## Importação de dados


In [24]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 37.7 MB/s 


In [144]:
# 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from unidecode import unidecode
import warnings
warnings.filterwarnings("ignore")

In [51]:
# Importação de dados
dataset = pd.read_csv('.../nova_base_vagas.csv')
dataset.head()

,Empresa,Vaga,Local,Nota,Link,Descrição,Sede,Tamanho,Fundação,Tipo,Receita,Oportunidades de carreira,Remuneração e benefícios,Cultura e valores,Alta liderança,Qualidade de vida,Recomendam para outras pessoas,Setor
0,Unisoma Computação,Analista de Data Science / Cientista de Dados,Campinas,3.5,https://www.glassdoor.com.br/partner/jobListin...,Oportunidade para quem tem conhecimento em mod...,"Campinas, Brasil",De 1 a 50 funcionários,1984.0,Empresa – capital fechado,De US$ 5 a US$ 25 milhões,2.7,3.5,3.7,3.5,4.0,74 %,NaN
1,Digio,Cientista de Dados Sênior,Trabalho remoto,4.4,https://www.glassdoor.com.br/partner/jobListin...,O objetivo da célula de Data Science do Digio ...,"Barueri, Brasil",De 201 a 500 funcionários,2013.0,Empresa – capital fechado,Desconhecido/Não se aplica,4.3,4.4,4.4,3.8,4.3,95 %,Financeiro
2,GRUPO AGROMERCANTIL,Líder Cientista de Dados,Uberlândia,5.0,https://www.glassdoor.com.br/partner/jobListin...,MISSÃO\nLiderar pessoas e projetos desenvolven...,"Uberlândia, Brasil",De 1 a 50 funcionários,NaN,Empresa – capital fechado,Desconhecido/Não se aplica,5.0,3.0,5.0,4.0,4.5,100 %,NaN
3,A3Data,Cientista de Dados Pleno e Sênior,Trabalho remoto,4.7,https://www.glassdoor.com.br/partner/jobListin...,A A3Data é uma consultoria referência em #Dado...,"Belo Horizonte, Brasil",De 51 a 200 funcionários,2014.0,Empresa – capital fechado,Desconhecido/Não se aplica,4.4,3.6,4.8,4.7,4.3,98 %,Tecnologia da informação
4,marketdata,Cientista de Dados Pleno,São Paulo,4.5,https://www.glassdoor.com.br/partner/jobListin...,"Quem Somos:\n\nA Marketdata, empresa do grupo ...","São Paulo, Brasil",De 201 a 500 funcionários,NaN,Subsidiária ou segmento de negócio,Desconhecido/Não se aplica,4.1,3.7,4.7,4.4,4.3,97 %,NaN


# Feature Engineering

## Funções


In [265]:
def filtro_0(string):
  """
  Filtro para remover somente a informação do cargo da vaga.
  ----
  string : string
    Texto a ser filtrado pelo regex
  ----
  Explicação do regex:
    (\w+) :  Seleciona uma palavra
    \s : Espaço 
    (?=) : Verifica se o grupo pode ser encontrado iniciando-se na posição que se encontra. 

  """

  filtro = re.search(r'(\w+)\s(\w+)(?=\s+dados)', string)
  primeira = filtro.group(1)
  segunda = filtro.group(2)

  if segunda != 'de':
    return f'{segunda} de dados'
  else:
    return f'{primeira} {segunda} dados'


## Tratamento de dados


In [185]:
# Filtragem Inicial: Manter ofertas de vagas que apresentam as palavras Dados ou Data.
dados = dataset.copy()
manter = dados.loc[(dados['Vaga'].str.lower().str.contains('dado')) | dados['Vaga'].str.lower().str.contains('data')].index 
remover = list(set(dados.index) - set(manter))
dados = dados.iloc[manter,:].reset_index(drop=True)

print(f'{" "*20} Dados Removidos\n')
dataset.iloc[remover,1].tolist()

                     Dados Removidos



['Técnico de Instrumentação Sênior - São João da Barra',
 'Técnico de Planejamento - Juruti',
 'Especialista de Business Intelligence | Campinas/ SP',
 'Programa de Estágio IBM Associates Brasil',
 'Analista de Comunicação']

In [186]:
# Remoção dos acentos das palavras
dados['Vaga'] = dados['Vaga'].apply(lambda x: unidecode(x))

# Remoção de marcadores
remover = ['(',')','-','|','[',']','/']
for icon in remover:
  dados['Vaga'] =dados['Vaga'].str.replace(icon,'')

# Substituir abreviações e passar tudo para o minúsculo 
substituir = [['SR','senior'],['Sr','senior'],['PL','pleno'],['II','pleno']]
for i in substituir:
  dados['Vaga'] = dados['Vaga'].str.replace(i[0], i[1])

dados['Vaga'] = dados['Vaga'].str.lower()

# Substituir o nome em inglês para português
dados['Vaga'] = dados['Vaga'].str.replace('data scientist','cientista de dados')

In [187]:
# Algumas vagas apresentam cargos disponiveis para mais de uma experiência,
# Então para uma análise mais precisa esses dados foram duplicados e separados
# Pelo nível de experiência.

remover = dados.loc[(dados['Vaga'].str.contains('pleno')) & dados['Vaga'].str.contains('senior')].index
dados_2 = dados.loc[(dados['Vaga'].str.contains('pleno')) & dados['Vaga'].str.contains('senior')]
dados_2.loc[:,'Vaga'] = dados_2['Vaga'].str.replace('pleno','').values
dados.loc[remover,'Vaga'] = dados.iloc[remover,1].str.replace('senior','')
dados = pd.concat([dados, dados_2], axis=0)

In [188]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90 entries, 0 to 82
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Empresa                         90 non-null     object 
 1   Vaga                            90 non-null     object 
 2   Local                           90 non-null     object 
 3   Nota                            84 non-null     float64
 4   Link                            90 non-null     object 
 5   Descrição                       90 non-null     object 
 6   Sede                            74 non-null     object 
 7   Tamanho                         78 non-null     object 
 8   Fundação                        59 non-null     float64
 9   Tipo                            78 non-null     object 
 10  Receita                         78 non-null     object 
 11  Oportunidades de carreira       77 non-null     float64
 12  Remuneração e benefícios        77 non

In [189]:
# Tratar dados númericos.
dados['Recomendam para outras pessoas'] = dados['Recomendam para outras pessoas'].str.replace('%','').str.replace('N/A','0').fillna('0').astype(int)

# Preenchendo vazios
preencher = dados.select_dtypes(include='object').columns
for colunas in preencher:
  dados[colunas] = dados[colunas].fillna('Não Informado')

In [190]:
dados.head()

,Empresa,Vaga,Local,Nota,Link,Descrição,Sede,Tamanho,Fundação,Tipo,Receita,Oportunidades de carreira,Remuneração e benefícios,Cultura e valores,Alta liderança,Qualidade de vida,Recomendam para outras pessoas,Setor
0,Unisoma Computação,analista de data science cientista de dados,Campinas,3.5,https://www.glassdoor.com.br/partner/jobListin...,Oportunidade para quem tem conhecimento em mod...,"Campinas, Brasil",De 1 a 50 funcionários,1984.0,Empresa – capital fechado,De US$ 5 a US$ 25 milhões,2.7,3.5,3.7,3.5,4.0,74,Não Informado
1,Digio,cientista de dados senior,Trabalho remoto,4.4,https://www.glassdoor.com.br/partner/jobListin...,O objetivo da célula de Data Science do Digio ...,"Barueri, Brasil",De 201 a 500 funcionários,2013.0,Empresa – capital fechado,Desconhecido/Não se aplica,4.3,4.4,4.4,3.8,4.3,95,Financeiro
2,GRUPO AGROMERCANTIL,lider cientista de dados,Uberlândia,5.0,https://www.glassdoor.com.br/partner/jobListin...,MISSÃO\nLiderar pessoas e projetos desenvolven...,"Uberlândia, Brasil",De 1 a 50 funcionários,NaN,Empresa – capital fechado,Desconhecido/Não se aplica,5.0,3.0,5.0,4.0,4.5,100,Não Informado
3,A3Data,cientista de dados pleno e,Trabalho remoto,4.7,https://www.glassdoor.com.br/partner/jobListin...,A A3Data é uma consultoria referência em #Dado...,"Belo Horizonte, Brasil",De 51 a 200 funcionários,2014.0,Empresa – capital fechado,Desconhecido/Não se aplica,4.4,3.6,4.8,4.7,4.3,98,Tecnologia da informação
4,marketdata,cientista de dados pleno,São Paulo,4.5,https://www.glassdoor.com.br/partner/jobListin...,"Quem Somos:\n\nA Marketdata, empresa do grupo ...","São Paulo, Brasil",De 201 a 500 funcionários,NaN,Subsidiária ou segmento de negócio,Desconhecido/Não se aplica,4.1,3.7,4.7,4.4,4.3,97,Não Informado


## Criação de Features


In [269]:
# Gerando uma coluna com o cargo da vaga
dados['Cargo'] = dados['Vaga'].apply(lambda x: filtro_0(x)).tolist()

In [294]:
# Nível de experiência exigido

dados['Experiência'] = dados['Cargo'].apply(lambda x: 'Não Informado')
filtrar = ['junior','senior','pleno']
informar = []
for palavra in filtrar:
  sub = dados.loc[dados['Vaga'].str.contains(palavra)].index
  dados.loc[sub,'Experiência'] = dados.loc[sub,'Experiência'].apply(lambda x: palavra)

In [299]:
dados.head()

,Empresa,Vaga,Local,Nota,Link,Descrição,Sede,Tamanho,Fundação,Tipo,Receita,Oportunidades de carreira,Remuneração e benefícios,Cultura e valores,Alta liderança,Qualidade de vida,Recomendam para outras pessoas,Setor,Cargo,Experiência
0,Unisoma Computação,analista de data science cientista de dados,Campinas,3.5,https://www.glassdoor.com.br/partner/jobListin...,Oportunidade para quem tem conhecimento em mod...,"Campinas, Brasil",De 1 a 50 funcionários,1984.0,Empresa – capital fechado,De US$ 5 a US$ 25 milhões,2.7,3.5,3.7,3.5,4.0,74,Não Informado,cientista de dados,Não Informado
1,Digio,cientista de dados senior,Trabalho remoto,4.4,https://www.glassdoor.com.br/partner/jobListin...,O objetivo da célula de Data Science do Digio ...,"Barueri, Brasil",De 201 a 500 funcionários,2013.0,Empresa – capital fechado,Desconhecido/Não se aplica,4.3,4.4,4.4,3.8,4.3,95,Financeiro,cientista de dados,senior
2,GRUPO AGROMERCANTIL,lider cientista de dados,Uberlândia,5.0,https://www.glassdoor.com.br/partner/jobListin...,MISSÃO\nLiderar pessoas e projetos desenvolven...,"Uberlândia, Brasil",De 1 a 50 funcionários,NaN,Empresa – capital fechado,Desconhecido/Não se aplica,5.0,3.0,5.0,4.0,4.5,100,Não Informado,cientista de dados,Não Informado
3,A3Data,cientista de dados pleno e,Trabalho remoto,4.7,https://www.glassdoor.com.br/partner/jobListin...,A A3Data é uma consultoria referência em #Dado...,"Belo Horizonte, Brasil",De 51 a 200 funcionários,2014.0,Empresa – capital fechado,Desconhecido/Não se aplica,4.4,3.6,4.8,4.7,4.3,98,Tecnologia da informação,cientista de dados,pleno
4,marketdata,cientista de dados pleno,São Paulo,4.5,https://www.glassdoor.com.br/partner/jobListin...,"Quem Somos:\n\nA Marketdata, empresa do grupo ...","São Paulo, Brasil",De 201 a 500 funcionários,NaN,Subsidiária ou segmento de negócio,Desconhecido/Não se aplica,4.1,3.7,4.7,4.4,4.3,97,Não Informado,cientista de dados,pleno


# Exportar Dados


In [303]:
dados.to_csv('/content/drive/MyDrive/Colab Assuntos Exemplos/Web Scraping/Ofertas de vagas/base_dados_tratado.csv')